# Obtained_data:

In [1]:
import numpy as np 

In [2]:
import pandas as pd
user_data=pd.read_csv("Final_User_Dataset.csv")
item_data=pd.read_csv("Final_Anime_Dataset.csv")
user_input=pd.read_csv("User_input.csv")



In [3]:
user_data.columns

Index(['user_id', 'Username', 'Gender', 'Birth_Year', 'Mean Score',
       'Completed', 'avg_Action', 'avg_Adventure', 'avg_Avant Garde',
       'avg_Award Winning', 'avg_Boys Love', 'avg_Comedy', 'avg_Drama',
       'avg_Ecchi', 'avg_Erotica', 'avg_Fantasy', 'avg_Girls Love',
       'avg_Gourmet', 'avg_Hentai', 'avg_Horror', 'avg_Mystery', 'avg_Romance',
       'avg_Sci-Fi', 'avg_Slice of Life', 'avg_Sports', 'avg_Supernatural',
       'avg_Suspense'],
      dtype='object')

In [4]:
item_data.columns

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status',
       'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
       'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL',
       'Genre_Action', 'Genre_Adventure', 'Genre_Avant Garde',
       'Genre_Award Winning', 'Genre_Boys Love', 'Genre_Comedy', 'Genre_Drama',
       'Genre_Ecchi', 'Genre_Erotica', 'Genre_Fantasy', 'Genre_Girls Love',
       'Genre_Gourmet', 'Genre_Hentai', 'Genre_Horror', 'Genre_Mystery',
       'Genre_Romance', 'Genre_Sci-Fi', 'Genre_Slice of Life', 'Genre_Sports',
       'Genre_Supernatural', 'Genre_Suspense', 'Genre_UNKNOWN', 'Type_Movie',
       'Type_Music', 'Type_ONA', 'Type_OVA', 'Type_Special', 'Type_TV',
       'Type_UNKNOWN', 'Rating_G - All Ages', 'Rating_PG - Children',
       'Rating_PG-13 - Teens 13 or older',
       'Rating_R - 17+ (violence & profanity)', 'Rating_R+ - Mil

In [6]:
item_data.drop(columns="Type", inplace=True)

In [7]:
from sklearn.preprocessing import OneHotEncoder

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop=None, dtype=int)

# Fit and transform the 'Gender' column
gender_encoded = encoder.fit_transform(user_data[['Gender']])

# Get the column names for the encoded features
gender_columns = encoder.get_feature_names_out(['Gender'])

# Create a DataFrame for the encoded features
gender_encoded_df = pd.DataFrame(gender_encoded, columns=gender_columns)

# Concatenate the one-hot encoded columns with the original DataFrame
user_data = pd.concat([user_data, gender_encoded_df], axis=1)

# Drop the original 'Gender' column if no longer needed
user_data.drop(columns=['Gender'], inplace=True)

# Display the updated DataFrame
print(user_data.head())

   user_id  Username  Birth_Year  Mean Score  Completed  avg_Action  \
0        1     Xinil        1985        7.37      233.0        7.68   
1       20    vondur        1988        8.06       94.0        8.02   
2       66    Hiromi        1990        7.53      148.0        7.49   
3       82    Achtor        1989        7.17      153.0        7.36   
4      112  luffykun        1983        8.77      125.0        8.60   

   avg_Adventure  avg_Avant Garde  avg_Award Winning  avg_Boys Love  ...  \
0           7.90              8.0               8.18            0.0  ...   
1           8.24              9.0               8.43            0.0  ...   
2           7.31              7.0               7.90            0.0  ...   
3           7.71              8.8               8.25            0.0  ...   
4           8.36              0.0               9.80            0.0  ...   

   avg_Mystery  avg_Romance  avg_Sci-Fi  avg_Slice of Life  avg_Sports  \
0         7.86         7.27        7.37   

In [8]:
user_data.shape

(71285, 29)

In [9]:
user_data.columns

Index(['user_id', 'Username', 'Birth_Year', 'Mean Score', 'Completed',
       'avg_Action', 'avg_Adventure', 'avg_Avant Garde', 'avg_Award Winning',
       'avg_Boys Love', 'avg_Comedy', 'avg_Drama', 'avg_Ecchi', 'avg_Erotica',
       'avg_Fantasy', 'avg_Girls Love', 'avg_Gourmet', 'avg_Hentai',
       'avg_Horror', 'avg_Mystery', 'avg_Romance', 'avg_Sci-Fi',
       'avg_Slice of Life', 'avg_Sports', 'avg_Supernatural', 'avg_Suspense',
       'Gender_Female', 'Gender_Male', 'Gender_Non-Binary'],
      dtype='object')

In [10]:
item_data.columns


Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Episodes', 'Aired', 'Premiered', 'Status', 'Producers',
       'Licensors', 'Studios', 'Source', 'Duration', 'Rating', 'Rank',
       'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL',
       'Genre_Action', 'Genre_Adventure', 'Genre_Avant Garde',
       'Genre_Award Winning', 'Genre_Boys Love', 'Genre_Comedy', 'Genre_Drama',
       'Genre_Ecchi', 'Genre_Erotica', 'Genre_Fantasy', 'Genre_Girls Love',
       'Genre_Gourmet', 'Genre_Hentai', 'Genre_Horror', 'Genre_Mystery',
       'Genre_Romance', 'Genre_Sci-Fi', 'Genre_Slice of Life', 'Genre_Sports',
       'Genre_Supernatural', 'Genre_Suspense', 'Genre_UNKNOWN', 'Type_Movie',
       'Type_Music', 'Type_ONA', 'Type_OVA', 'Type_Special', 'Type_TV',
       'Type_UNKNOWN', 'Rating_G - All Ages', 'Rating_PG - Children',
       'Rating_PG-13 - Teens 13 or older',
       'Rating_R - 17+ (violence & profanity)', 'Rating_R+ - Mild Nudity

In [11]:

item_data.isna().sum()

anime_id                                    0
Name                                        0
English name                                0
Other name                                  0
Score                                    9213
Genres                                      0
Synopsis                                    0
Episodes                                    0
Aired                                       0
Premiered                                   0
Status                                      0
Producers                                   0
Licensors                                   0
Studios                                     0
Source                                      0
Duration                                    0
Rating                                      0
Rank                                        0
Popularity                                  0
Favorites                                   0
Scored By                                   0
Members                           

In [12]:
filtered_item_data_df = item_data.drop(columns=["Genres","Episodes","Rating_UNKNOWN",'Producers','Aired',"Premiered","Status","Studios","Licensors","Duration"])

In [13]:
filtered_item_data_df.columns

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Synopsis',
       'Source', 'Rating', 'Rank', 'Popularity', 'Favorites', 'Scored By',
       'Members', 'Image URL', 'Genre_Action', 'Genre_Adventure',
       'Genre_Avant Garde', 'Genre_Award Winning', 'Genre_Boys Love',
       'Genre_Comedy', 'Genre_Drama', 'Genre_Ecchi', 'Genre_Erotica',
       'Genre_Fantasy', 'Genre_Girls Love', 'Genre_Gourmet', 'Genre_Hentai',
       'Genre_Horror', 'Genre_Mystery', 'Genre_Romance', 'Genre_Sci-Fi',
       'Genre_Slice of Life', 'Genre_Sports', 'Genre_Supernatural',
       'Genre_Suspense', 'Genre_UNKNOWN', 'Type_Movie', 'Type_Music',
       'Type_ONA', 'Type_OVA', 'Type_Special', 'Type_TV', 'Type_UNKNOWN',
       'Rating_G - All Ages', 'Rating_PG - Children',
       'Rating_PG-13 - Teens 13 or older',
       'Rating_R - 17+ (violence & profanity)', 'Rating_R+ - Mild Nudity',
       'Rating_Rx - Hentai', 'Release_Year'],
      dtype='object')

In [14]:
filtered_item_data_df.shape

(24905, 50)

In [15]:
# Keep rows where both 'Score' and 'Release_Year' are not null
filtered_item_data_df = filtered_item_data_df[filtered_item_data_df["Score"].notna() & filtered_item_data_df["Release_Year"].notna()]

In [16]:
print(filtered_item_data_df.isna().sum())

anime_id                                 0
Name                                     0
English name                             0
Other name                               0
Score                                    0
Synopsis                                 0
Source                                   0
Rating                                   0
Rank                                     0
Popularity                               0
Favorites                                0
Scored By                                0
Members                                  0
Image URL                                0
Genre_Action                             0
Genre_Adventure                          0
Genre_Avant Garde                        0
Genre_Award Winning                      0
Genre_Boys Love                          0
Genre_Comedy                             0
Genre_Drama                              0
Genre_Ecchi                              0
Genre_Erotica                            0
Genre_Fanta

In [17]:
filtered_item_data_df.shape

(14754, 50)

In [20]:
user_data.columns

Index(['user_id', 'Username', 'Birth_Year', 'Mean Score', 'Completed',
       'avg_Action', 'avg_Adventure', 'avg_Avant Garde', 'avg_Award Winning',
       'avg_Boys Love', 'avg_Comedy', 'avg_Drama', 'avg_Ecchi', 'avg_Erotica',
       'avg_Fantasy', 'avg_Girls Love', 'avg_Gourmet', 'avg_Hentai',
       'avg_Horror', 'avg_Mystery', 'avg_Romance', 'avg_Sci-Fi',
       'avg_Slice of Life', 'avg_Sports', 'avg_Supernatural', 'avg_Suspense',
       'Gender_Female', 'Gender_Male', 'Gender_Non-Binary'],
      dtype='object')

In [18]:
filtered_item_data_df.columns

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Synopsis',
       'Source', 'Rating', 'Rank', 'Popularity', 'Favorites', 'Scored By',
       'Members', 'Image URL', 'Genre_Action', 'Genre_Adventure',
       'Genre_Avant Garde', 'Genre_Award Winning', 'Genre_Boys Love',
       'Genre_Comedy', 'Genre_Drama', 'Genre_Ecchi', 'Genre_Erotica',
       'Genre_Fantasy', 'Genre_Girls Love', 'Genre_Gourmet', 'Genre_Hentai',
       'Genre_Horror', 'Genre_Mystery', 'Genre_Romance', 'Genre_Sci-Fi',
       'Genre_Slice of Life', 'Genre_Sports', 'Genre_Supernatural',
       'Genre_Suspense', 'Genre_UNKNOWN', 'Type_Movie', 'Type_Music',
       'Type_ONA', 'Type_OVA', 'Type_Special', 'Type_TV', 'Type_UNKNOWN',
       'Rating_G - All Ages', 'Rating_PG - Children',
       'Rating_PG-13 - Teens 13 or older',
       'Rating_R - 17+ (violence & profanity)', 'Rating_R+ - Mild Nudity',
       'Rating_Rx - Hentai', 'Release_Year'],
      dtype='object')

In [19]:
user_input.columns

Index(['Username', 'Gender', 'Mean Score', 'Completed', 'Birth_Year',
       'user_id', 'anime_id', 'Anime Title', 'rating', 'Genres',
       'Genre_Action', 'Genre_Adventure', 'Genre_Avant Garde',
       'Genre_Award Winning', 'Genre_Boys Love', 'Genre_Comedy', 'Genre_Drama',
       'Genre_Ecchi', 'Genre_Erotica', 'Genre_Fantasy', 'Genre_Girls Love',
       'Genre_Gourmet', 'Genre_Hentai', 'Genre_Horror', 'Genre_Mystery',
       'Genre_Romance', 'Genre_Sci-Fi', 'Genre_Slice of Life', 'Genre_Sports',
       'Genre_Supernatural', 'Genre_Suspense'],
      dtype='object')

In [20]:
def prepare_training_data(user_input_df, user_data_df, filtered_item_data_df):
    """
    Prepares training data for a neural network by merging dataframes and extracting
    user features (X_U), anime features (X_A), and ratings (Y).
    
    Parameters:
    -----------
    user_input_df : DataFrame
        Contains user ratings for anime with columns including user_id, anime_id, rating
    user_data_df : DataFrame
        Contains user demographic data and average genre ratings
    filtered_item_data_df : DataFrame
        Contains anime metadata including genres, types, and ratings
        
    Returns:
    --------
    dict
        A dictionary containing:
        - 'X_U': User features array
        - 'X_A': Anime features array
        - 'Y': Rating values array
        - 'X_U_columns': List of column names for user features
        - 'X_A_columns': List of column names for anime features
        - 'Y_column': Name of the rating column
        - 'X_U_indices': List of indices for user features
        - 'X_A_indices': List of indices for anime features
        - 'Y_index': Index of the rating column
        - 'merged_df': The merged and cleaned dataframe
    """
    import numpy as np
    
    # Step 1: Merge dataframes
    merged_df = user_input_df.merge(user_data_df, on='user_id')
    merged_df = merged_df.merge(filtered_item_data_df, on='anime_id')
    
    # Step 2: Identify columns to drop (non-numerical or identifiers)
    columns_to_drop = ['Username', 'user_id', 'anime_id', 'Anime Title', 
                      'Name', 'English name', 'Other name', 'Synopsis', 
                      'Source', 'Image URL']
    
    # Only drop columns that actually exist in the merged dataframe
    columns_to_drop = [col for col in columns_to_drop if col in merged_df.columns]
    
    # Drop identified columns
    cleaned_df = merged_df.drop(columns_to_drop, axis=1)
    
    # Step 3: Define columns for each component
    X_U_columns = ['Birth_Year', 'Mean Score', 'Completed',
                  'avg_Action', 'avg_Adventure', 'avg_Avant Garde', 'avg_Award Winning',
                  'avg_Boys Love', 'avg_Comedy', 'avg_Drama', 'avg_Ecchi', 'avg_Erotica',
                  'avg_Fantasy', 'avg_Girls Love', 'avg_Gourmet', 'avg_Hentai',
                  'avg_Horror', 'avg_Mystery', 'avg_Romance', 'avg_Sci-Fi',
                  'avg_Slice of Life', 'avg_Sports', 'avg_Supernatural', 'avg_Suspense',
                  'Gender_Female', 'Gender_Male', 'Gender_Non-Binary']
    
    X_A_columns = ['Score', 'Rating', 'Rank', 'Popularity', 'Favorites', 'Scored By',
                  'Members', 'Release_Year',
                  'Genre_Action', 'Genre_Adventure', 'Genre_Avant Garde', 'Genre_Award Winning',
                  'Genre_Boys Love', 'Genre_Comedy', 'Genre_Drama', 'Genre_Ecchi', 'Genre_Erotica',
                  'Genre_Fantasy', 'Genre_Girls Love', 'Genre_Gourmet', 'Genre_Hentai',
                  'Genre_Horror', 'Genre_Mystery', 'Genre_Romance', 'Genre_Sci-Fi',
                  'Genre_Slice of Life', 'Genre_Sports', 'Genre_Supernatural',
                  'Genre_Suspense', 'Genre_UNKNOWN',
                  'Type_Movie', 'Type_Music', 'Type_ONA', 'Type_OVA', 'Type_Special', 'Type_TV', 'Type_UNKNOWN',
                  'Rating_G - All Ages', 'Rating_PG - Children', 'Rating_PG-13 - Teens 13 or older',
                  'Rating_R - 17+ (violence & profanity)', 'Rating_R+ - Mild Nudity', 'Rating_Rx - Hentai']
    
    Y_column = 'rating'
    
    # Filter to include only columns that exist in the cleaned dataframe
    X_U_columns = [col for col in X_U_columns if col in cleaned_df.columns]
    X_A_columns = [col for col in X_A_columns if col in cleaned_df.columns]
    
    # Verify rating column exists
    if Y_column not in cleaned_df.columns:
        raise ValueError(f"Rating column '{Y_column}' not found in merged dataframe")
    
    # Step 4: Get indices for each component
    X_U_indices = [cleaned_df.columns.get_loc(col) for col in X_U_columns]
    X_A_indices = [cleaned_df.columns.get_loc(col) for col in X_A_columns]
    Y_index = cleaned_df.columns.get_loc(Y_column)
    
    # Step 5: Extract features using indices
    X_U = cleaned_df.iloc[:, X_U_indices].values
    X_A = cleaned_df.iloc[:, X_A_indices].values
    Y = cleaned_df.iloc[:, Y_index].values
    
    # Return all relevant data and metadata
    return {
        'X_U': X_U,
        'X_A': X_A,
        'Y': Y,
        'X_U_columns': X_U_columns,
        'X_A_columns': X_A_columns,
        'Y_column': Y_column,
        'X_U_indices': X_U_indices,
        'X_A_indices': X_A_indices,
        'Y_index': Y_index,
        'merged_df': cleaned_df
    }

In [22]:
result = prepare_training_data(user_input, user_data, filtered_item_data_df)

# Access the prepared data
X_U = result['X_U']  # User features
X_A = result['X_A']  # Anime features
Y = result['Y']      # Ratings

# You can also access indices if needed
X_U_indices = result['X_U_indices']
X_A_indices = result['X_A_indices']

In [23]:
X_U.shape

(8443619, 24)

In [24]:
X_A.shape

(8443619, 22)

In [25]:
Y.shape

(8443619,)

In [26]:
Y[:10]

array([9, 7, 5, 8, 8, 8, 7, 5, 6, 7], dtype=int64)

# Neural Network for implementing Content Based Filtering:

In [27]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models


In [4]:
tf.random.set_seed(42)
num_outputs=32
# Define the model for the user
# The model is a simple feedforward neural network with 4 hidden layers
User_NN=tf.keras.models.Sequential([
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(units=num_outputs,activation="linear")    
])

item_NN=tf.keras.models.Sequential([
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(units=num_outputs,activation="linear") 
])

# Create the user input and point input layers
user_input = tf.keras.Input(shape=(num_outputs,), name='user_input')
vu=User_NN(input=user_input)
vu=tf.linalg.norm(vu,axis=1)

# Create the item input and point input layers
item_input = tf.keras.Input(shape=(num_outputs,), name='item_input')
va=item_NN(input=item_input)
va=tf.linalg.norm(va,axis=1)

#Compute the dot product of the user and item vectors
dot_product = tf.keras.layers.Dot(axes=1)([vu, va])

model=tf.keras.Model(inputs=[user_input, item_input], outputs=dot_product)

model.summary()


TypeError: missing a required argument: 'inputs'

In [ ]:
# Compiling the model
      
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')